In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut,GeocoderUnavailable
from pandas.core.common import flatten
import ast
from tqdm import tqdm
import requests_cache


In [598]:
full_year = pd.read_csv("../../input-data/temp-data.csv")

In [599]:
full_year

,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,...,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
0,0,Wrestling with age and a case of idea theft.,https://www.nytimes.com/2024/09/01/business/he...,Wrestling with age and a case of idea theft.,"Send questions about the office, money, career...",BU,3.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Help! I’m ‘Older’ and on the Job Hun...,...,2024-09-01T04:01:07+0000,article,SundayBusiness,Business Day,"{'original': 'By Anna Holmes', 'person': [{'fi...",News,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,1280,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,NaN
1,1,"Grueling shifts, abuse from the public and sub...",https://www.nytimes.com/2024/09/01/world/asia/...,"Grueling shifts, abuse from the public and sub...",Exhausted doctors resting in crowded on-call r...,A,4.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Worked to the Bone, India’s Doctors ...",...,2024-09-01T04:01:25+0000,article,Foreign,World,{'original': 'By Anupreeta Das and Pragati K.B...,News,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,1310,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,Asia Pacific
2,2,"About a quarter of the residents of Maracaibo,...",https://www.nytimes.com/2024/09/01/world/ameri...,"About a quarter of the residents of Maracaibo,...",It was once a thriving metropolis in the heart...,A,1.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What Happens When Half a Million Peo...,...,2024-09-01T04:01:27+0000,article,Foreign,World,{'original': 'By Frances Robles and Marian Car...,News,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,1460,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,Americas
3,3,Israel and Hamas agreed to pause the war to pe...,https://www.nytimes.com/2024/09/01/world/middl...,Israel and Hamas agreed to pause the war to pe...,Health workers on Sunday began a polio vaccina...,A,1.0,The New York Times,[],"{'main': 'In Race Against Polio, Gaza Begins V...",...,2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Bilal Shbair, Erika Solomon a...",News,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,1473,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,Middle East
4,4,The prospect of a strong performance by the fa...,https://www.nytimes.com/2024/09/01/world/europ...,The prospect of a strong performance by the fa...,"On Sunday, voters in the eastern German states...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What to Watch For in East German Sta...,...,2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Christopher F. Schuetze', 'pe...",News,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,812,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48691,48691,The first Black officer to achieve the rank of...,https://www.nytimes.com/2024/08/31/us/arthur-j...,The first Black officer to achieve the rank of...,"Arthur J. Gregg, the first African American Ar...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Arthur J. Gregg, Trailblazing Army O...",...,2024-08-31T14:57:12+0000,article,Obits,U.S.,"{'original': 'By Trip Gabriel', 'person': [{'f...",Obituary (Obit),nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,1019,nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,NaN
48692,48692,"In a reversal, two senior U.S. military offici...",https://www.nytimes.com/2024/08/31/world/europ...,"In a reversal, two senior U.S. military offici...",As hundreds of Russian missiles and drones str...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'As Ukraine Mourns a Pilot’s Death, J...",...,2024-08-31T14:57:53+0000,article,Foreign,Wor

In [600]:
full_year.columns

Index(['Unnamed: 0', 'abstract', 'web_url', 'snippet', 'lead_paragraph',
       'print_section', 'print_page', 'source', 'multimedia', 'headline',
       'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name',
       'byline', 'type_of_material', '_id', 'word_count', 'uri',
       'subsection_name'],
      dtype='object')

In [601]:
full_year_essential = full_year[["_id", "section_name", "keywords", "pub_date"]].copy()

In [602]:
full_year_essential['keywords'] = full_year_essential['keywords'].apply(ast.literal_eval)

In [603]:
full_year_essential = full_year_essential.explode("keywords")

In [604]:
full_year_essential["keyword_type"] = full_year_essential["keywords"].get("name")

In [605]:
full_year_essential["keyword"] = full_year_essential["keywords"].get("value")

In [606]:
full_year_essential["keyword_type"] = full_year_essential["keywords"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
full_year_essential["keyword"] = full_year_essential["keywords"].apply(lambda x: x.get("value") if isinstance(x, dict) else None)
full_year_essential["keyword_rank"] = full_year_essential["keywords"].apply(lambda x: x.get("rank") if isinstance(x, dict) else None)

In [623]:
full_year_essential

,_id,section_name,keywords,pub_date,keyword_type,keyword,keyword_rank
0,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,Business Day,"{'name': 'subject', 'value': 'Careers and Prof...",2024-09-01T04:01:07+0000,subject,Careers and Professions,1.0
0,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,Business Day,"{'name': 'subject', 'value': 'Hiring and Promo...",2024-09-01T04:01:07+0000,subject,Hiring and Promotion,2.0
0,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,Business Day,"{'name': 'subject', 'value': 'Content Type: Se...",2024-09-01T04:01:07+0000,subject,Content Type: Service,3.0
0,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,Business Day,"{'name': 'subject', 'value': 'Workplace Enviro...",2024-09-01T04:01:07+0000,subject,Workplace Environment,4.0
0,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,Business Day,"{'name': 'subject', 'value': 'your-feed-selfca...",2024-09-01T04:01:07+0000,subject,your-feed-selfcare,5.0
...,...,...,...,...,...,...,...
48695,nyt://article/0bca89dd-a1a7-5022-8192-9812f680...,New York,"{'name': 'subject', 'value': 'United States Op...",2024-08-31T15:42:26+0000,subject,United States Open (Tennis),1.0
48695,nyt://article/0bca89dd-a1a7-5022-8192-9812f680...,New York,"{'name': 'subject', 'value': 'Content Type: Pe...",2024-08-31T15:42:26+0000,subject,Content Type: Personal Profile,3.0
48695,nyt://article/0bca89dd-a1a7-5022-8192-9812f680...,New York,"{'name': 'subject', 'value': 'Tennis', 'rank':...",2024-08-31T15:42:26+0000,subject,Tennis,6.0
48695,nyt://article/0bca89dd-a1a7-5022-8192-9812f680...,New York,"{'name': 'glocations', 'value': 'Netherlands',...",2024-08-31T15:42:26+0000,glocations,Netherlands,7.0


In [607]:
locations = full_year_essential[full_year_essential["keyword_type"] == "glocations"]

In [ ]:
locations = locations["keyword"].unique()

In [ ]:
locations

In [ ]:
places_df = pd.DataFrame(locations, columns=["location"])

In [ ]:
places_df

In [ ]:
places_df['country'] = places_df['location'].apply(lambda x: x[x.find("(")+1:x.find(")")] if "(" in x and ")" in x else x)

## Geolocate individual locations

In [ ]:
geolocator = Nominatim(user_agent="geo_locator")

In [ ]:
# Function to get country
def get_country(location):
    try:
        geo = geolocator.geocode(location, exactly_one=True, language='en', addressdetails=False)
        if geo:
            return geo.address.split(",")[-1].strip()
        else:
            return "Not found"
    except GeocoderTimedOut:
        return "Timeout"
    except GeocoderUnavailable:
        return "Unavailable"

In [ ]:
places_df['retrieved_country'] = places_df.apply(lambda x: get_country(x["country"]), axis=1)

In [ ]:
places_df.to_csv("../../input-data/nyt_retrived_countries.csv")

## Clean edge cases

In [573]:
retrieved_countries = pd.read_csv("../../input-data/nyt_retrived_countries.csv")

In [574]:
retrieved_countries = retrieved_countries.drop(labels="Unnamed: 0", axis=1)

Masking all US states with ambigous 2 letter code (not real ISO code, will default to "United States" as country).

In [575]:
us_states_mask = (retrieved_countries['country'].str.len() == 2) | (retrieved_countries['country'].str.len() == 3) | (retrieved_countries['country'] == "Los Angeles, Calif") | (retrieved_countries['country'] == "Miami, Fla")

In [576]:
retrieved_countries.loc[us_states_mask, "retrieved_country"] = "United States"

In [577]:
retrieved_countries

,location,country,retrieved_country
0,India,India,India
1,Maracaibo (Venezuela),Venezuela,Venezuela
2,Venezuela,Venezuela,Venezuela
3,Israel,Israel,Israel
4,Germany,Germany,Germany
...,...,...,...
3169,Yanji (China),China,China
3170,"West Side Highway (Manhattan, NY)","Manhattan, NY",United States
3171,County Clare (Ireland),Ireland,Ireland
3172,County Limerick (Ireland),Ireland,Ireland


Masking all rows where geopy was not successful at finding a match.

In [578]:
undefined_mask = (retrieved_countries['retrieved_country'] == "Not found") | (retrieved_countries['retrieved_country'] == "Unavailable")

Some of them do not have a match because the US state name is not recognized

In [579]:
list_of_states = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']

In [580]:
undefined_countries = retrieved_countries.loc[undefined_mask]

In [581]:
undefined_countries

,location,country,retrieved_country
28,Minnesota,Minnesota,Unavailable
80,"Far East, South and Southeast Asia and Pacific...","Far East, South and Southeast Asia and Pacific...",Not found
131,Kansas,Kansas,Unavailable
145,Arizona,Arizona,Unavailable
172,London (England),England,Unavailable
203,Tiananmen Square (Beijing),Beijing,Unavailable
247,Maryland,Maryland,Unavailable
299,Oregon,Oregon,Unavailable
459,USSR (Former Soviet Union),Former Soviet Union,Not found
670,Cyclades Islands,Cyclades Islands,Not found


In [582]:
undefined_us_states = [country for country in undefined_countries["country"] if country.upper() in list_of_states]

In [583]:
undefined_us_states

['Minnesota',
 'Kansas',
 'Arizona',
 'Maryland',
 'Oregon',
 'Vermont',
 'Alaska',
 'Ohio']

In [584]:
mask_undefined_us = retrieved_countries["country"].isin(undefined_us_states)
retrieved_countries.loc[mask_undefined_us, "retrieved_country"] = "United States"

In [588]:
undefined_mask = (retrieved_countries['retrieved_country'] == "Not found") | (retrieved_countries['retrieved_country'] == "Unavailable")
undefined_countries = retrieved_countries.loc[undefined_mask]
undefined_countries

,location,country,retrieved_country
80,"Far East, South and Southeast Asia and Pacific...","Far East, South and Southeast Asia and Pacific...",Not found
172,London (England),England,Unavailable
203,Tiananmen Square (Beijing),Beijing,Unavailable
459,USSR (Former Soviet Union),Former Soviet Union,Not found
670,Cyclades Islands,Cyclades Islands,Not found
772,Panama Canal and Canal Zone,Panama Canal and Canal Zone,Not found
823,Sahara Desert,Sahara Desert,Unavailable
873,Darien Gap,Darien Gap,Not found
886,ANTARCTIC REGIONS,ANTARCTIC REGIONS,Not found
1079,"Valladolid (Yucatan Penninsula, Mexico)","Yucatan Penninsula, Mexico",Not found


In [586]:
missing_countries = ["Armenia", "St Vincent", "Lebanon", "Mali"]

In [ ]:
mask_undefined_countries = retrieved_countries["country"].isin(missing_countries)
retrieved_countries.loc[mask_undefined_countries, "retrieved_country"] = retrieved_countries.loc[mask_undefined_countries, "country"] 

In [592]:
siberia_mask = retrieved_countries["country"] == "Siberia"
retrieved_countries.loc[siberia_mask, "retrieved_country"] = "Russia"
retrieved_countries

,location,country,retrieved_country
0,India,India,India
1,Maracaibo (Venezuela),Venezuela,Venezuela
2,Venezuela,Venezuela,Venezuela
3,Israel,Israel,Israel
4,Germany,Germany,Germany
...,...,...,...
3169,Yanji (China),China,China
3170,"West Side Highway (Manhattan, NY)","Manhattan, NY",United States
3171,County Clare (Ireland),Ireland,Ireland
3172,County Limerick (Ireland),Ireland,Ireland


In [593]:
west_bank_mask = retrieved_countries["country"] == "West Bank"
retrieved_countries.loc[west_bank_mask, "retrieved_country"] = "Palestinian Territory"

In [594]:
clean_countries = retrieved_countries.drop(retrieved_countries.loc[undefined_mask].index)

In [595]:
clean_countries

,location,country,retrieved_country
0,India,India,India
1,Maracaibo (Venezuela),Venezuela,Venezuela
2,Venezuela,Venezuela,Venezuela
3,Israel,Israel,Israel
4,Germany,Germany,Germany
...,...,...,...
3169,Yanji (China),China,China
3170,"West Side Highway (Manhattan, NY)","Manhattan, NY",United States
3171,County Clare (Ireland),Ireland,Ireland
3172,County Limerick (Ireland),Ireland,Ireland


## Retrieve article ids from original dataset

In [643]:
data = []
for index, place in clean_countries.iterrows():
    place_mask = full_year_essential["keyword"] == place["location"]
    place_coverage = full_year_essential.loc[place_mask, "_id"]
    data.append((place["location"], place["retrieved_country"], len(place_coverage.values), place_coverage.to_list()))

In [639]:
places_and_ids_df = pd.DataFrame(data, columns=["place_keyword", "country", "count_of_articles", "ids_of_articles"])

In [644]:
places_and_ids_df

,place_keyword,country,count_of_articles,ids_of_articles
0,India,India,234,[nyt://article/aeabc262-aeb0-5423-a7ac-8bb664c...
1,Maracaibo (Venezuela),Venezuela,1,[nyt://article/42c0d0f2-ea62-5d2b-8eba-baa0418...
2,Venezuela,Venezuela,103,[nyt://article/42c0d0f2-ea62-5d2b-8eba-baa0418...
3,Israel,Israel,2143,[nyt://article/6393c6c3-0e1f-5494-925d-165e7aa...
4,Germany,Germany,241,[nyt://article/fe046102-78e5-530d-89e0-59ff09c...
...,...,...,...,...
3143,Yanji (China),China,1,[nyt://article/ab3117c5-2380-5493-b5a2-9a275ff...
3144,"West Side Highway (Manhattan, NY)",United States,1,[nyt://article/bcae809a-57ae-5b4a-a574-168659e...
3145,County Clare (Ireland),Ireland,1,[nyt://article/441056ed-eb8a-5ad0-a0c1-a2ef99d...
3146,County Limerick (Ireland),Ireland,1,[nyt://article/441056ed-eb8a-5ad0-a0c1-a2ef99d...


In [ ]:
places_and_ids_df

## Group by country and chain ids of articles together

In [659]:
countries_and_unique_ids = places_and_ids_df.copy()

In [660]:
from itertools import chain
general_countries = (countries_and_unique_ids.groupby('country', as_index=False)['ids_of_articles']
         .agg(lambda x: list(chain.from_iterable(x)))
       )

## Remove duplicates from id list and count number of articles for each country

In [ ]:
general_countries["ids_of_articles"] = general_countries["ids_of_articles"].apply(lambda x: list(set(x)))

In [ ]:
general_countries["count_of_articles"] = general_countries["ids_of_articles"].apply(lambda x: len(x))

## Retrieve coordinates for each country

In [664]:
geolocator = Nominatim(user_agent="geo_lookup")
session = requests_cache.CachedSession("geopy_cache", expire_after=86400)  # Cache for 1 day
tqdm.pandas()

In [669]:
def get_coordinates(location_name, country_code=None):
    query = f"{location_name}, {country_code}" if country_code else location_name
    try:
        location = geolocator.geocode(query, timeout=10)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error for {query}: {e}")
    return None, None

In [670]:
general_countries[["Latitude", "Longitude"]] = general_countries.progress_apply(
    lambda row: get_coordinates(row["country"]), axis=1, result_type="expand"
)

100%|██████████| 230/230 [03:49<00:00,  1.00it/s]


In [672]:
general_countries["outlet"] = "NYT"

In [673]:
general_countries

,country,ids_of_articles,count_of_articles,Latitude,Longitude,outlet
0,Adriatic Sea,[nyt://article/6c5919fe-452a-532d-8234-f698a33...,1,43.702151,14.667946,NYT
1,Aegean Sea,[nyt://article/a0eec7f0-3d63-5711-9700-9bce1b3...,2,38.062228,25.720589,NYT
2,Afghanistan,[nyt://article/afe74c10-e930-55fc-9186-2163146...,89,33.768006,66.238514,NYT
3,Africa,[nyt://article/c2ae5cf2-31fc-5962-98b0-53d4cd1...,199,11.502434,17.757812,NYT
4,Albania,[nyt://article/68d601c6-5e2e-5646-8c14-c73d2ef...,20,5.758765,-73.915162,NYT
...,...,...,...,...,...,...
225,Vietnam,[nyt://interactive/7c53d0e8-f08c-5155-b878-6ae...,51,15.926666,107.965086,NYT
226,Yellow Sea,[nyt://article/593fcdd3-0e07-58b1-9a5d-eec8cd8...,1,36.000000,124.000000,NYT
227,Yemen,[nyt://article/bcfcf8e1-77de-5c59-8df3-725ae54...,90,16.347124,47.891527,NYT
228,Zambia,[nyt://article/415aea4c-2180-5418-ba50-e6bad35...,11,-14.518912,27.558988,NYT


In [674]:
general_countries.to_csv("../../data/places/coverage_by_country.csv")